# Wikipedia 한국어 데이터셋 다운로드

이 노트북에서는 Hugging Face Datasets에서 한국어 Wikipedia 데이터셋을 다운로드하고 분석합니다.


In [ ]:
# 필요한 라이브러리 설치 및 임포트
%pip install datasets

from datasets import load_dataset
import pandas as pd
import json
from tqdm import tqdm


In [ ]:
# Wikipedia 한국어 데이터셋 다운로드
print("=== Wikipedia 한국어 데이터셋 다운로드 시작 ===")

try:
    # 데이터셋 로드 (처음에는 시간이 오래 걸릴 수 있습니다)
    print("데이터셋 로딩 중... (처음 실행 시 다운로드가 필요합니다)")
    dataset = load_dataset("wikimedia/wikipedia", "20231101.ko")
    
    print(f"✅ 데이터셋 로드 완료!")
    print(f"📊 데이터셋 정보:")
    print(f"   - 훈련 데이터: {len(dataset['train']):,}개 문서")
    print(f"   - 컬럼: {dataset['train'].column_names}")
    
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    dataset = None


In [ ]:
# 샘플 데이터 확인
if dataset:
    print("📝 샘플 문서들:")
    
    for i in range(3):
        sample = dataset['train'][i]
        print(f"\n--- 문서 {i+1} ---")
        print(f"ID: {sample.get('id', 'N/A')}")
        print(f"제목: {sample.get('title', 'N/A')}")
        print(f"텍스트 길이: {len(sample.get('text', '')):,}자")
        print(f"텍스트 미리보기: {sample.get('text', '')[:300]}...")


In [ ]:
# 샘플 데이터를 로컬에 저장
if dataset:
    print("=== 샘플 데이터 저장 중 ===")
    
    # 1000개 샘플 추출
    num_samples = 1000
    sample_data = dataset['train'].select(range(min(num_samples, len(dataset['train']))))
    
    # CSV로 저장
    sample_data.to_csv('wikipedia_ko_sample.csv')
    print(f"✅ CSV 파일 저장 완료: wikipedia_ko_sample.csv ({len(sample_data)}개 문서)")
    
    # JSON으로 저장
    sample_data.to_json('wikipedia_ko_sample.json')
    print(f"✅ JSON 파일 저장 완료: wikipedia_ko_sample.json ({len(sample_data)}개 문서)")
    
    # 통계 정보
    print(f"\n📊 샘플 데이터 통계:")
    print(f"   - 총 문서 수: {len(sample_data):,}개")
    print(f"   - 평균 텍스트 길이: {sum(len(doc['text']) for doc in sample_data) // len(sample_data):,}자")
    print(f"   - 최대 텍스트 길이: {max(len(doc['text']) for doc in sample_data):,}자")
    print(f"   - 최소 텍스트 길이: {min(len(doc['text']) for doc in sample_data):,}자")


In [ ]:
# 특정 주제로 필터링 예제
if dataset:
    print("=== 특정 주제로 필터링 ===")
    
    # '인공지능' 관련 문서 찾기
    ai_docs = []
    for i, doc in enumerate(tqdm(dataset['train'], desc="검색 중...")):
        if '인공지능' in doc.get('title', '') or '인공지능' in doc.get('text', ''):
            ai_docs.append(doc)
            if len(ai_docs) >= 5:  # 5개만 찾기
                break
    
    print(f"\n🤖 '인공지능' 관련 문서 {len(ai_docs)}개 발견:")
    for i, doc in enumerate(ai_docs):
        print(f"{i+1}. {doc.get('title', 'N/A')}")
        print(f"   텍스트 길이: {len(doc.get('text', '')):,}자")
        print(f"   미리보기: {doc.get('text', '')[:100]}...")
        print()


## 데이터셋 사용법

💡 이제 Wikipedia 한국어 데이터셋을 사용할 수 있습니다:

### 1. 전체 데이터셋 사용
```python
dataset = load_dataset('wikimedia/wikipedia', '20231101.ko')
docs = dataset['train']
```

### 2. 특정 문서 접근
```python
doc = dataset['train'][0]  # 첫 번째 문서
title = doc['title']
text = doc['text']
```

### 3. 샘플 데이터 사용
```python
import pandas as pd
df = pd.read_csv('wikipedia_ko_sample.csv')
```

### 4. RAG 시스템에 활용
- 텍스트를 청크로 분할
- 임베딩 생성
- 벡터 데이터베이스에 저장
- 검색 및 생성에 활용
